In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import itertools

import networkx as nx
import time
from operator import itemgetter


In [ ]:
def yield_cmp_len1(*args, **kwargs):
    global round
    v = set(G_bar.nodes()).difference(G_bar[reachable[0]])
    G_bar.remove_nodes_from(v)
    yield v
    

In [ ]:
def yield_cmp_len1_nonreach(*args, **kwargs):
    global round
    v = set(G_bar.nodes()).difference(G_bar[reachable[0]])
    G_bar.remove_nodes_from(v)
    yield v
    

In [ ]:
def score(H,cmp_nodes):
    degree = H.degree(cmp_nodes)
    sum_dg1 = sum([d[1] for d in degree ])

    h = nx.induced_subgraph(H,cmp_nodes)
    degree = h.degree(cmp_nodes)
    sum_dg2 = sum([d[1] for d in degree ])
    
    return (sum_dg2/2)/(sum_dg1-sum_dg2)


In [ ]:
def yield_cmp_len_greater_1(*args, **kwargs):
    global round
    f1 = False
    
    node_set1 = set()
    node_set1.update(nbr for n in reachable for nbr in G_bar[n] if nbr in non_reachable)
    if node_set1 == set(non_reachable):
        f1 = True
    else:
        cmp_nodes1 = G_bar.nodes() ^ node_set1
        score1 = score(G,cmp_nodes1)
    
    node_set2 = set()
    node_set2.update(nbr for n in non_reachable for nbr in G_bar[n] if nbr in reachable)
    if node_set2 == set(reachable):
        if f1:
            degree = G_bar.degree()
            sorted_nodes = sorted(degree, key=itemgetter(1))
            mindeg = sorted_nodes[0][0]
            v = set(G_bar.nodes()).difference(G_bar[mindeg])
            #v = set(G[mindeg])
            #v.add(mindeg)
            G_bar.remove_nodes_from(v)
            yield v
        else:
            G_bar.remove_nodes_from(cmp_nodes1)
            yield cmp_nodes1
        
    elif f1:
        cmp_nodes2 = G_bar.nodes() ^ node_set2
        G_bar.remove_nodes_from(cmp_nodes2)
        yield cmp_nodes2
    else:
        cmp_nodes2 = G_bar.nodes() ^ node_set2
        score2 = score(G,cmp_nodes2)
        if score1 > score2:
            G_bar.remove_nodes_from(cmp_nodes1)
            yield cmp_nodes1
        else:
            G_bar.remove_nodes_from(cmp_nodes2)
            yield cmp_nodes2


In [ ]:
def min_cut(*args, **kwargs):
    global round
    while len(G_bar.nodes()) > 1:
        if len(G_bar.nodes()) <= 3:
            yield set(G_bar.nodes())
            break
        elif not nx.is_connected(G_bar):
            yield set(G_bar.nodes())
            G_bar.clear()
        else:
            global reachable, non_reachable
            cut_value, partition = nx.stoer_wagner(G_bar)
            reachable, non_reachable = partition
            if len(reachable) == 1:
                yield yield_cmp_len1().__next__()
            elif len(non_reachable) == 1:
                yield yield_cmp_len1_nonreach().__next__()
            else:
                yield yield_cmp_len_greater_1().__next__()
    else:
        yield set(G_bar.nodes())

In [ ]:
if __name__ == '__main__':
    global reachable, non_reachable, G_bar, G, node_deg1
    start_time = time.time()
    rounds = 1
    G_bar = nx.Graph()
    G = nx.read_weighted_edgelist("FileName.txt", create_using = nx.Graph(), nodetype = str)
    G_cmp = nx.read_weighted_edgelist("WeightedGraphComplement.txt", create_using = nx.Graph(), nodetype = str)
    for g in nx.connected_components(G):

        if len(g)>3:
            G_bar = nx.induced_subgraph(G_cmp,g).copy()
            result = min_cut()
        else:
            result = [g]
        print('The program is started......')
        
        i = 1
        edges = 0
        with open('Output_clusters.txt', 'a') as f:
            if G_bar.number_of_edges()>0:
                for r in result:
                    if r:
                        sbg = nx.induced_subgraph(G,r)
                        sbg_cmp = list(nx.connected_components(sbg))
                        if len(sbg_cmp) > 1:
                            for cmp in sbg_cmp:
                                f.writelines("%s " % c for c in cmp)
                                f.write("\n")
                            edges += len(sbg.edges())
                        else: 
                            f.writelines("%s " % v for v in r)
                            f.write("\n")
                            edges += len(sbg.edges())
            else:
                sbg = nx.induced_subgraph(G,g)
                sbg_cmp = list(nx.connected_components(sbg))
                print(sbg_cmp)
                if len(sbg_cmp) > 1:
                    for cmp in sbg_cmp:
                        f.writelines("%s " % c for c in cmp)
                        f.write("\n")
                    edges += len(sbg.edges())
                else: 
                    f.writelines("%s " % v for v in g)
                    f.write("\n")
                    edges += len(sbg.edges())
                print('round after one result in a file %s' % rounds)
                rounds += 1
        if not nx.is_empty(G_bar):
            G_bar.clear()
        #G_bar.clear()
    print('Number of removed edges is: %s' % (len(G.edges()) - edges))
    
    f_name = 'Output_clusters.txt'
    with open(f_name) as f:
        cmplx = f.readlines()
    cmplx.sort(key=len, reverse=True)
    cmplx = [x.strip() for x in cmplx] 
    cmplx = {tuple(x.split(' ')) for x in cmplx}
    
    cmplx = list(cmplx)
    cmplx.sort(key=len, reverse=True)
    
    with open(f_name, 'w') as f:
        for item in cmplx:
            for node in item:
                f.write("%s " % node)
            f.write("\n")


    print("--- %s seconds ---" % (time.time() - start_time))
